# API Nakala

## Packages

In [ ]:
using CSV
using DataFrames
using ImageSegmentation
using HTTP
using JSON
using Dates
using FileIO

## Identifiants

In [ ]:
path = @__DIR__
credentials = CSV.read(joinpath(path, "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [ ]:
apitest = true

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

## Chercher un fichier précis dans une donnée

In [ ]:
title = "Z1J434"
lang = ""
firstImage = 1
lastImage = 2

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]


url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 100,
  :titleSearch => title,
  :titleSearchLang => lang
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")
# il peut y avoir plusieurs résultats
data = datas[1]
identifier = get(datas[1], "identifier", "")
files = get(data, "files", "")

## Création d'un array pour les images

In [ ]:
urls = Vector()
for i in firstImage:lastImage
  file = files[i]
  fileName = get(file, "name", "unknown")

  fileIdentifier = get(file, "sha1", "unknown")
  fileUrl = joinpath(apiurl, "data", identifier, fileIdentifier)

  img = Dict(
    "url" => fileUrl,
    "apiurl" => apiurl,
    "dataIdentifier" => identifier,
    "fileIdentifier" => fileIdentifier,
    "label" => fileName
  )

  push!(urls, img)
end

urls

## Fonction Split()

In [ ]:
function split(label, file, s)
  middle = s[2]/2
  l = floor(Int, middle+100)
  r = floor(Int, middle-100)
  leftPart = @view file[ :, 1:l]
  rightPart = @view file[ :, r:s[2]]
  save(string(splitext(label)[1], "-l", splitext(label)[2]), leftPart)
  save(string(splitext(label)[1], "-r", splitext(label)[2]), rightPart)
end

In [ ]:
for img in urls
  label = get(img, "label", "unknown")
  file = download(get(img, "url", "")) |> load
  s = size(file)
  
  file = rot180(file)

  if s[1] < s[2]
    split(label, file, s)
  else
    save(label, file)
  end

end

## Spliiit


In [ ]:
function spliiit(image)
    
    img = download(get(image, "url", "")) |> load
    
    segments = felzenszwalb(img, 5000, 300000) # le deuxième argument permet de supprimer les segments les plus petits
    
    segMap = labels_map(segments)
    
    pixel = sort(segment_pixel_count(segments); byvalue=true)
    v = collect(values(pixel))
    k = collect(keys(pixel))
    l = length(collect(values(pixel)))
    
    vec = Vector()
    for i in l-1:l
        coordinates = findall(x -> x == k[i], segMap)

        ymin, ymax = extrema(c -> c[1], coordinates)
        h = ymax - ymin

        xmin, xmax = extrema(c -> c[2], coordinates)
        w = xmax - xmin
        
        region = joinpath(get(image, "apiurl", ""), "iiif", get(image, "dataIdentifier", ""), get(image, "fileIdentifier", ""), "$xmin,$ymin,$w,$h/full/0/default.jpg")
        push!(vec, region)

    end   
    
    vec
end


In [ ]:
t = Vector() 
for img in urls
    push!(t, spliiit(img))
end
t